# Analyze SDG Data availability

## Load libraries

In [7]:
import pandas as pd
import os
import csv
import json
import statistics

import numpy as np
from pathlib import Path

dir_path = os.path.dirname(os.path.realpath('__file__'))
print(dir_path)

# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

C:\Users\L.GonzalezMorales\Documents\Windows Projects\fis4sdg_python\notebooks\reports


### Read M49 countries and groups

In [9]:
ghana = pd.read_excel('../../data/external/m49_availability.xlsx', sheet_name='Ghana')
regions = pd.read_excel('../../data/external/m49_availability.xlsx', sheet_name='Regions')

ghana.head()
regions.head()

,countryCode,countryName,LDC,LLDC,SIDS,Development,regionCode,regionName
0,288,Ghana,NaN,NaN,NaN,Developing,202,Sub-Saharan Africa


,Code,SDG_Region_Name
0,1,World
1,747,Northern Africa and Western Asia
2,202,Sub-Saharan Africa
3,419,Latin America and the Caribbean
4,513,Europe and Northern America


### Read metadata

In [10]:
metadatafile = 'metadata_2020.Q1.G.02.json'

with open('../../data/external/'+metadatafile) as json_file:
    metadata = json.load(json_file)

metadata[0].keys()
metadata[0]['targets'][0].keys()
metadata[0]['targets'][0]['indicators'][0].keys()

dict_keys(['code', 'labelEN', 'descEN', 'labelES', 'descES', 'labelFR', 'descFR', 'labelRU', 'descRU', 'labelZN', 'descZN', 'hex', 'rgb', 'ColorScheme', 'thumbnail', 'targets'])

dict_keys(['code', 'labelEN', 'descEN', 'labelES', 'descES', 'labelFR', 'descFR', 'labelRU', 'descRU', 'labelZN', 'descZN', 'indicators'])

dict_keys(['code', 'reference', 'labelEN', 'descEN', 'labelES', 'descES', 'labelFR', 'descFR', 'labelRU', 'descRU', 'labelZN', 'descZN', 'series'])

### Functions

#### 1. Read dataset

In [11]:
def read_sdg_dataset(csvfile):
    return pd.read_csv('../../data/processed/2020.Q1.G.02/'+csvfile) 


#### 2. Build csv file name

In [12]:
def csv_file(indicator, series):
    return ('Indicator_' + indicator.replace('.', '_') + '__Series_' + series + '.csv')

#### Identify disaggregations

In [13]:
def get_disaggregations(data):
   
    dim_columns = list(data.columns)
    exclude_cols = ['goal_code','goal_labelEN','goal_descEN','target_code','target_descEN',
                'indicator_code','indicator_reference','indicator_descEN',
                'series_release','series_tags','series','seriesDescription',
                'geoAreaCode','geoAreaName','level','parentCode','parentName','type',
                'X','Y','ISO3','UN_Member','Country_Profile','timeCoverage','geoInfoUrl',
                'years','min_year','max_year','n_years',
                'unitsCode','unitsDesc','reportingTypeCode','reportingTypeDesc',
                'latest_value','basePeriod','valueDetails','footnotes','sources','timeDetails','nature']

    d = [i for i in dim_columns if i not in exclude_cols and not i.startswith('value_') and not i.startswith('upper') and not i.startswith('lower') and not i.endswith('_desc')] 
    
    #return [sub.replace('_code', '') for sub in d] 
    return d


### Print list of goals, targets, indicators, series

In [14]:
df_list = []


for g in metadata:
    #if g['code'] != '6':
    #    continue
    for t in g['targets']:
        for i in t['indicators']:
            for s in i['series']:
                #if s['code'] != 'SI_POV_EMP1':
                #    continue
                    
                #print(f"Indicator {i['reference']}, Series {s['code']}: {s['description']}.")
                
                f = csv_file(i['reference'], s['code'])
                      
                data = read_sdg_dataset(f)                      
                
                print(f"disaggregations for {s['code']}:")
                get_disaggregations(data)
                
                country_data_cols = ['geoAreaCode', 'geoAreaName','years', 'min_year', 'max_year', 'n_years']
                country_data_cols.extend(get_disaggregations(data))
                #country_data_cols
                      
                ## Join countries and data:
                      
                country_data = pd.merge(ghana, 
                                        data[country_data_cols], 
                                        how='inner', 
                                        left_on=['countryCode'], 
                                        right_on=['geoAreaCode']).reset_index()
                      
                x = country_data.copy()
                      
                #----Check number of years and latest year available

                country_availability = x.groupby(['countryCode', 'countryName'], as_index=False).agg({'max_year':['max'],
                                                                                                          'n_years':['max', 'count']})      
                country_availability.columns = ['countryCode', 'countryName','max_year', 'n_years', 'n_timeSeries']
                      
                #----Check number of disaggregations in the data set
                      
                country_availability['n_dimensions']  = len(get_disaggregations(data)) 
                
                #----Check average number levels/values per dimension
                      
                if len(get_disaggregations(data)) > 0:
                      country_availability['values_per_dim']  =  country_availability['n_timeSeries'] ** (1/len(get_disaggregations(data)))  
                else:
                      country_availability['values_per_dim']  =  None
                
                #----Check disaggregation by sex
                      
                if 'sex_code' in get_disaggregations(data):
                    xx = data[['geoAreaCode']][data['sex_code']=='F'].drop_duplicates()
                    xx = xx['geoAreaCode'].tolist()
                else:
                    xx = []
                
                country_availability['data_for_female'] = np.where(country_availability['countryCode'].isin(xx), 1, 0)
                
                #----add regional groupings
                      
                country_availability = pd.merge(countries,
                                                country_availability,
                                                how='left',
                                                on = ['countryCode']
                                                ).reset_index()
                      
                      
                country_availability['Goal'] = '\''+ g['code']
                country_availability['Target'] = '\'' + t['code']
                country_availability['Indicator_ref'] = i['reference']
                country_availability['Series_code'] = s['code']
                country_availability['Series_desc'] = s['description']
                #country_availability['disaggregations_list'] = get_disaggregations(data)
                      
                #country_availability
                      
                ## ---------------Aggregate by region------------------
                      
                x = country_availability.copy()
                
                if len(get_disaggregations(data)) > 0:
                    country_availability_by_region = x.groupby(['Goal','Target','Indicator_ref','Series_code', 'Series_desc', 'regionCode', 'regionName'], 
                                                               as_index=False).agg({'max_year':['count','median'],
                                                                                    'n_years':['median'],
                                                                                    'n_timeSeries':['median'],
                                                                                    'n_dimensions':['median'],
                                                                                    'values_per_dim':['median'],
                                                                                    'data_for_female':['mean']})
                else:
                    country_availability_by_region = x.groupby(['Goal','Target','Indicator_ref','Series_code','Series_desc','regionCode', 'regionName'], 
                                                               as_index=False).agg({'max_year':['count','median'],
                                                                                    'n_years':['median'],
                                                                                    'n_timeSeries':['median'],
                                                                                    'n_dimensions':['median']})
                      
                    country_availability_by_region['values_per_dim'] = None
                    country_availability_by_region['data_for_female'] = None

                country_availability_by_region.columns = ['Goal','Target','Indicator_ref','Series_code','Series_desc','regionCode', 'regionName',
                                                          'n_countries_available', 'median_max_year', 'median_n_years', 'median_n_timeSeries',
                                                          'n_dimensions', 'median_values_per_dim', 'mean_data_for_female'
                                                         ]
                      
                      
                country_availability_by_region = pd.merge(country_availability_by_region, 
                                                       x.groupby(['regionCode'],as_index=False)['countryCode'].agg('count'),
                                                       how='left',
                                                       on = ['regionCode']).reset_index()
                country_availability_by_region.rename(columns={'countryCode': 'n_countries_total'}, inplace=True)
                
                #country_availability_by_region
                      
                ## ---------------Aggregate by series ------------------  
                      
                      
                x = country_availability.copy()
                
                
                if len(get_disaggregations(data)) > 0:
                    country_availability_world = x.groupby(['Goal','Target','Indicator_ref','Series_code','Series_desc'], 
                                                           as_index=False).agg({'max_year':['count','median'],
                                                                                'n_years':['median'],
                                                                                'n_timeSeries':['median'],
                                                                                'n_dimensions':['median'],
                                                                                'values_per_dim':['median'],
                                                                                'data_for_female':['mean']})
                    
                else:
                    country_availability_world = x.groupby(['Goal','Target','Indicator_ref','Series_code', 'Series_desc'], 
                                                           as_index=False).agg({'max_year':['count','median'],
                                                                                'n_years':['median'],
                                                                                'n_timeSeries':['median'],
                                                                                'n_dimensions':['median']}) 
                    country_availability_world['values_per_dim'] = None
                    country_availability_world['data_for_female'] = None
                
                country_availability_world.columns = ['Goal','Target','Indicator_ref','Series_code','Series_desc',
                                                          'n_countries_available', 'median_max_year', 'median_n_years', 'median_n_timeSeries',
                                                          'n_dimensions', 'median_values_per_dim', 'mean_data_for_female'
                                                         ]
                      
                      
                country_availability_world = pd.merge(country_availability_world, 
                                                       x.groupby(['Series_code','Series_desc'],as_index=False)['countryCode'].agg('count'),
                                                       how='left',
                                                       on = ['Series_code','Series_desc']).reset_index()
                country_availability_world.rename(columns={'countryCode': 'n_countries_total'}, inplace=True)
                
                country_availability_world['regionCode'] = 1
                country_availability_world['regionName'] = 'World'
                      
                frames = [country_availability_world,country_availability_by_region ]
                x = pd.concat(frames)
                      
                x.to_csv('../../data/availability/2020.Q1.G.02/ghana_'+ f)
                      
                df_list.append(x)
                      
availability_by_series_and_region = pd.concat(df_list)
                      
availability_by_series_and_region.to_csv('../../data/availability/2020.Q1.G.02/ghana_Availability_by_series.csv')

print('finished!')
                      

disaggregations for SI_POV_DAY1:


[]

disaggregations for SI_POV_EMP1:


['age_code', 'sex_code']

disaggregations for SI_POV_NAHC:


...

disaggregations for SG_REG_BRTH90N:


[]

disaggregations for SG_REG_DETH75N:


[]

finished!


### Availability by indicator

In [9]:
x = availability_by_series_and_region

x.median_values_per_dim = pd.to_numeric(x.median_values_per_dim)
x.mean_data_for_female= pd.to_numeric(x.mean_data_for_female)

availability_by_indicator_and_region = x.groupby(['Goal','Target','Indicator_ref', 'regionCode', 'regionName'], 
                                                  as_index=False).agg({'n_countries_available':['mean'],
                                                                      'median_max_year':['mean'],
                                                                      'median_n_years':['mean'],
                                                                      'median_n_timeSeries':['mean'],
                                                                      'n_dimensions':['mean'],
                                                                      'median_values_per_dim':['mean'],
                                                                      'mean_data_for_female':['mean'],
                                                                      'n_countries_total':['mean']})

availability_by_indicator_and_region.columns = ['Goal',
'Target',
'Indicator_ref',
'regionCode',
'regionName',
'avg_n_countries_available',
'avg_median_max_year',
'avg_median_n_years',
'avg_median_n_timeSeries',
'avg_n_dimensions',
'avg_median_values_per_dim',
'mean_data_for_female',
'n_countries_total']

availability_by_indicator_and_region.to_csv('../../data/availability/2020.Q1.G.02/Availability_by_indicator.csv')

print('finished!')
                     
       

finished!


In [11]:
x = availability_by_indicator_and_region

x.median_values_per_dim = pd.to_numeric(x.avg_median_values_per_dim)
x.mean_data_for_female= pd.to_numeric(x.mean_data_for_female)

availability_by_target_and_region = x.groupby(['Goal','Target', 'regionCode', 'regionName'], 
                                                  as_index=False).agg({'avg_n_countries_available':['mean'],
                                                                      'avg_median_max_year':['mean'],
                                                                      'avg_median_n_years':['mean'],
                                                                      'avg_median_n_timeSeries':['mean'],
                                                                      'avg_n_dimensions':['mean'],
                                                                      'avg_median_values_per_dim':['mean'],
                                                                      'mean_data_for_female':['mean'],
                                                                      'n_countries_total':['mean']})


availability_by_target_and_region.columns = ['Goal',
'Target',
'regionCode',
'regionName',
'avg_n_countries_available',
'avg_median_max_year',
'avg_median_n_years',
'avg_median_n_timeSeries',
'avg_n_dimensions',
'avg_median_values_per_dim',
'mean_data_for_female',
'n_countries_total']

availability_by_target_and_region.to_csv('../../data/availability/2020.Q1.G.02/Availability_by_target.csv')

print('finished!')
                    

finished!


In [13]:
x = availability_by_target_and_region

x.median_values_per_dim = pd.to_numeric(x.avg_median_values_per_dim)
x.mean_data_for_female= pd.to_numeric(x.mean_data_for_female)

availability_by_goal_and_region = x.groupby(['Goal','regionCode', 'regionName'], 
                                                  as_index=False).agg({'avg_n_countries_available':['mean'],
                                                                      'avg_median_max_year':['mean'],
                                                                      'avg_median_n_years':['mean'],
                                                                      'avg_median_n_timeSeries':['mean'],
                                                                      'avg_n_dimensions':['mean'],
                                                                      'avg_median_values_per_dim':['mean'],
                                                                      'mean_data_for_female':['mean'],
                                                                      'n_countries_total':['mean']})


availability_by_goal_and_region.columns = ['Goal',
'regionCode',
'regionName',
'avg_n_countries_available',
'avg_median_max_year',
'avg_median_n_years',
'avg_median_n_timeSeries',
'avg_n_dimensions',
'avg_median_values_per_dim',
'mean_data_for_female',
'n_countries_total']

availability_by_goal_and_region.to_csv('../../data/availability/2020.Q1.G.02/Availability_by_goal.csv')

print('finished!')
                    

finished!
